In [19]:
import pandas as pd
import numpy as np
import openpyxl
from datetime import datetime

In [20]:
filename = "LGZ67563259-ReadStoredValues_BillingDataProfile.xlsx"
serial_no = filename[3:].split("-")[0].strip()
serial_no

'67563259'

In [21]:
# Read the Excel file
df = pd.read_excel(filename)
df = df.drop(index=0).reset_index(drop=True)

In [22]:
#df['0-0:1.0.0'] = pd.to_date(df['0-0:1.0.0'], errors='coerce')
# df['0-0:1.0.0'] = pd.to_datetime(df['0-0:1.0.0']).dt.normalize()
# df['month'] = pd.to_datetime(df['0-0:1.0.0']).dt.month_name()
# df['year'] = pd.to_datetime(df['0-0:1.0.0']).dt.year
#df.dtypes

In [23]:
#df[['month','year']].head(2)

In [24]:
df.columns

Index(['#', '0-0:1.0.0', '1-0:0.1.0', '1-1:1.9.1 [kWh]', '1-1:1.9.2 [kWh]',
       '1-1:1.9.3 [kWh]', '1-1:2.9.1 [kWh]', '1-1:2.9.2 [kWh]',
       '1-1:2.9.3 [kWh]', '1-1:2.8.0 [kWh]', '1-1:10.8.0 [kVAh]',
       '1-1:4.8.0 [kvarh]', '1-1:1.8.0 [kWh]', '1-1:9.8.0 [kVAh]',
       '1-1:3.8.0 [kvarh]', '1-1:9.6.1;2 [MVA]', '1-1:9.6.1;5',
       '1-1:9.6.2;2 [MVA]', '1-1:9.6.2;5', '1-1:9.6.3;2 [MVA]', '1-1:9.6.3;5',
       '1-1:10.6.1;2 [MVA]', '1-1:10.6.1;5', '1-1:10.6.2;2 [MVA]',
       '1-1:10.6.2;5', '1-1:10.6.3;2 [MVA]', '1-1:10.6.3;5', '1-1:13.0.0'],
      dtype='object')

In [25]:
# columns_names = ['0-0:1.0.0', '1-0:0.1.0', '1-1:1.9.1 [kWh]', '1-1:1.9.2 [kWh]',
#        '1-1:1.9.3 [kWh]', '1-1:2.9.1 [kWh]', '1-1:2.9.2 [kWh]',
#        '1-1:2.9.3 [kWh]', '1-1:2.8.0 [kWh]', '1-1:10.8.0 [kVAh]',
#        '1-1:4.8.0 [kvarh]', '1-1:1.8.0 [kWh]', '1-1:9.8.0 [kVAh]',
#        '1-1:3.8.0 [kvarh]', '1-1:9.6.1;2 [MVA]', '1-1:9.6.1;5',
#        '1-1:9.6.2;2 [MVA]', '1-1:9.6.2;5', '1-1:9.6.3;2 [MVA]', '1-1:9.6.3;5',
#        '1-1:10.6.1;2 [MVA]', '1-1:10.6.1;5', '1-1:10.6.2;2 [MVA]',
#        '1-1:10.6.2;5', '1-1:10.6.3;2 [MVA]', '1-1:10.6.3;5', '1-1:13.0.0',
#        'month', 'year']

In [26]:
df = df.drop(df.columns[0], axis=1)
units = ['kWh', 'kVAh','kvarh','MVA']
max_dem_time = ['9.6.1;','9.6.2;','9.6.3;']
#print(df.columns)
for column in df.columns.to_list()[1:]:
    if any(substring in column for substring in units):
        new_column_name = column.split(":", 1)[1].split(' ')[0].split(';')[0]
        #print(column)
        df = df.rename(columns={column: new_column_name})

    elif any(substring in column for substring in max_dem_time):
        new_column_name = f"{column.split(":", 1)[1].split(';')[0]}_MDT"
        #print(column)
        df = df.rename(columns={column: new_column_name})
    

df.columns

Index(['0-0:1.0.0', '1-0:0.1.0', '1.9.1', '1.9.2', '1.9.3', '2.9.1', '2.9.2',
       '2.9.3', '2.8.0', '10.8.0', '4.8.0', '1.8.0', '9.8.0', '3.8.0', '9.6.1',
       '9.6.1_MDT', '9.6.2', '9.6.2_MDT', '9.6.3', '9.6.3_MDT', '10.6.1',
       '1-1:10.6.1;5', '10.6.2', '1-1:10.6.2;5', '10.6.3', '1-1:10.6.3;5',
       '1-1:13.0.0'],
      dtype='object')

In [27]:
df['9.6.3_MDT'] = df['9.6.3_MDT'].str.split('(', n=1).str[0]
df['9.6.2_MDT'] = df['9.6.2_MDT'].str.split('(', n=1).str[0]
df['9.6.1_MDT'] = df['9.6.1_MDT'].str.split('(', n=1).str[0]

In [28]:
df['9.6.3_MDT'] = pd.to_datetime(df['9.6.3_MDT'])
df['9.6.2_MDT'] = pd.to_datetime(df['9.6.2_MDT'])
df['9.6.1_MDT'] = pd.to_datetime(df['9.6.1_MDT'])

In [29]:
new_df_columns = {'1.9.1':'rate1', '1.9.2':'rate2', '1.9.3':'rate3', '2.9.1':'rate4', '2.9.2':'rate5',
       '2.9.3':'rate6', '2.8.0':'cumulative_export', '10.8.0':'apparent_export', 
       '4.8.0':'reactive_export', '1.8.0':'cumulative_import', '9.8.0':'apparent_import', 
       '3.8.0':'reactive_import', '9.6.1':'max_dem1','9.6.1_MDT':'max_dem1_datetime',
         '9.6.2':'max_dem2', '9.6.2_MDT':'max_dem2_datetime', '9.6.3':'max_dem3', 
         '9.6.3_MDT':'max_dem3_datetime'}
df = df.rename(columns=new_df_columns)
df = df.rename(columns={'0-0:1.0.0':'billing_period'})
new_df_columns = ['rate1','rate2','rate3','rate4','rate5','rate6','cumulative_export','apparent_export', 
                  'reactive_export','cumulative_import','apparent_import','reactive_import', 
                'max_dem1','max_dem1_datetime','max_dem2','max_dem2_datetime','max_dem3','max_dem3_datetime','billing_period']
df = df[new_df_columns]
df.head(3)

,rate1,rate2,rate3,rate4,rate5,rate6,cumulative_export,apparent_export,reactive_export,cumulative_import,apparent_import,reactive_import,max_dem1,max_dem1_datetime,max_dem2,max_dem2_datetime,max_dem3,max_dem3_datetime,billing_period
0,8747700.0,5406100.0,3628100.0,0.0,0.0,0.0,21.5,909.87,2011105.70,1.272814e+08,1.329063e+08,33087876.14,38.83,2025-01-05 15:30:00,38.94,2025-01-11 21:00:00,37.79,2025-01-14 01:00:00,2025-02-01 00:00:00
1,8194100.0,5388900.0,3318400.0,0.0,0.0,0.0,21.5,909.87,2010942.58,1.089328e+08,1.136917e+08,27654275.31,38.52,2024-12-11 13:30:00,39.94,2024-12-06 20:00:00,29.14,2024-12-17 00:30:00,2025-01-01 00:00:00
2,6801300.0,4534000.0,2760000.0,0.0,0.0,0.0,21.5,909.87,2010015.38,9.104120e+07,9.489531e+07,22085009.27,35.98,2024-11-25 16:30:00,37.60,2024-11-25 20:00:00,25.62,2024-11-30 00:30:00,2024-12-01 00:00:00


In [30]:
df['serial_no'] = serial_no 
df['reading_date'] = datetime.now().replace(second=0, microsecond=0)
df['vt_ratio'] = pd.NA  
df['ct_ratio'] = pd.NA  
df['no_of_resets'] = pd.NA  
df['date_of_last_reset'] = pd.NA  
df['programing_count'] = pd.NA  

In [31]:
df

,rate1,rate2,rate3,rate4,rate5,rate6,cumulative_export,apparent_export,reactive_export,cumulative_import,...,max_dem3,max_dem3_datetime,billing_period,serial_no,reading_date,vt_ratio,ct_ratio,no_of_resets,date_of_last_reset,programing_count
0,8747700.0,5406100.0,3628100.0,0.0,0.0,0.0,21.50,909.87,2011105.70,1.272814e+08,...,37.79,2025-01-14 01:00:00,2025-02-01 00:00:00,67563259,2025-03-09 23:54:00,<NA>,<NA>,<NA>,<NA>,<NA>
1,8194100.0,5388900.0,3318400.0,0.0,0.0,0.0,21.50,909.87,2010942.58,1.089328e+08,...,29.14,2024-12-17 00:30:00,2025-01-01 00:00:00,67563259,2025-03-09 23:54:00,<NA>,<NA>,<NA>,<NA>,<NA>
2,6801300.0,4534000.0,2760000.0,0.0,0.0,0.0,21.50,909.87,2010015.38,9.104120e+07,...,25.62,2024-11-30 00:30:00,2024-12-01 00:00:00,67563259,2025-03-09 23:54:00,<NA>,<NA>,<NA>,<NA>,<NA>
3,7209000.0,4590800.0,2970800.0,0.0,0.0,0.0,20.09,828.33,2008428.36,7.694585e+07,...,24.64,2024-10-29 00:30:00,2024-11-01 00:00:00,67563259,2025-03-09 23:54:00,<NA>,<NA>,<NA>,<NA>,<NA>
4,7112200.0,4711800.0,2893300.0,0.0,0.0,0.0,20.09,828.33,2003276.23,6.217517e+07,...,26.01,2024-09-11 00:30:00,2024-10-01 00:00:00,67563259,2025-03-09 23:54:00,<NA>,<NA>,<NA>,<NA>,<NA>
5,7761800.0,4709400.0,3040300.0,0.0,0.0,0.0,20.09,828.14,2002833.71,4.745802e+07,...,25.80,2024-08-22 00:30:00,2024-09-01 00:00:00,67563259,2025-03-09 23:54:00,<NA>,<NA>,<NA>,<NA>,<NA>
6,7837700.0,5007800.0,3197600.0,0.0,0.0,0.0,19.35,506.16,1180389.66,3.194650e+07,...,31.05,2024-07-05 00:30:00,2024-08-01 00:00:00,67563259,2025-03-09 23:54:00,<NA>,<NA>,<NA>,<NA>,<NA>
7,7490600.0,4873500.0,3028200.0,0.0,0.0,0.0,0.00,0.48,762356.65,1.590340e+07,...,27.72,2024-06-11 06:00:00,2024-07-01 00:00:00,67563259,2025-03-09 23:54:00,<NA>,<NA>,<NA>,<NA>,<NA>
8,282800.0,141000.0,86700.0,0.0,0.0,0.0,0.00,0.48,0.03,5.110039e+05,...,31.96,2024-05-31 03:30:00,2024-06-01 00:00:00,67563259,2025-03-09 23:54:00,<NA>,<NA>,<NA>,<NA>,<NA>
9,0.0,0.0,400.0,0.0,0.0,0.0,0.00,0.48,0.03,4.967200e+02,...,1.06,2024-05-31 02:41:54,2024-05-31 02:41:54,67563259,2025-03-09 23:54:00,<NA>,<NA>,<NA>,<NA>,<NA>


In [32]:
df.columns

Index(['rate1', 'rate2', 'rate3', 'rate4', 'rate5', 'rate6',
       'cumulative_export', 'apparent_export', 'reactive_export',
       'cumulative_import', 'apparent_import', 'reactive_import', 'max_dem1',
       'max_dem1_datetime', 'max_dem2', 'max_dem2_datetime', 'max_dem3',
       'max_dem3_datetime', 'billing_period', 'serial_no', 'reading_date',
       'vt_ratio', 'ct_ratio', 'no_of_resets', 'date_of_last_reset',
       'programing_count'],
      dtype='object')